# 1. Considerando los logs de acciones realizadas sobre ítems, mostrar el top 10 de ids de ítems que fueron afectados por mayor cantidad usuarios distintos (*)

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
import pandas as pd
import numpy as np
import os

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [3]:
#df = pd.read_csv('Wikipedia dataset 2c2021/redirect_list.csv')
#df.to_parquet('redirect_list.parquet')
#del df
logs_ori = sqlContext.read.parquet('Wikipedia dataset 2c2021/logs.parquet').rdd
logs_ori.take(5)

[Row(item_id=1, timestamp='2004-12-23T07:10:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo'),
 Row(item_id=2, timestamp='2004-12-23T07:14:09Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'", logtype='delete', action='delete', title='Rapunzel'),
 Row(item_id=3, timestamp='2004-12-23T07:26:10Z', contributor_username='Sabbut', contributor_id=2075.0, contributor_ip=None, comment="el contenido era: '#REDIRECT [[Flash]]'", logtype='delete', action='delete', title='Macromedia Flash'),
 Row(item_id=4, timestamp='2004-12-23T07:55:09Z', contributor_username='Sanbec', contributor_id=7856.0, contributor_ip=None, comment='Sin información de origen ni de 

In [4]:
#df2 = pd.read_csv('Wikipedia dataset 2c2021/contents.csv')
#df2.to_parquet('contents.parquet')
#del df2
contents_ori = sqlContext.read.parquet('Wikipedia dataset 2c2021/contents.parquet').rdd
contents_ori.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor_username='Lojwe

In [5]:
logs = logs_ori.map(lambda x: (x[8],(x[2] if x[2] is not None else x[4]))).filter(lambda x: (x[0] is not None) & (x[1] is not None))

In [6]:
contents = contents_ori.map(lambda x: (x[0],1)).filter(lambda x: (x[0] is not None))

In [7]:
contents.take(2)

[('Wikipedia:Artículos solicitados', 1), ('Andorra', 1)]

In [8]:
logs.take(2)

[('Verificacionismo', 'Sabbut'), ('Rapunzel', 'Sabbut')]

In [9]:
logs.count()

12186830

In [10]:
logs = logs.distinct()

In [11]:
logs.count()

11652041

In [12]:
merge = logs.join(contents)
merge.take(2)

[('Iglesia Parroquial de San Pedro y San Pablo', ('CEM-air', 1)),
 ('Categoría:Médicos por país y especialidad', ('Mercenario97', 1))]

In [13]:
merge = merge.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)

In [14]:
merge.take(2)

[('Iglesia Parroquial de San Pedro y San Pablo', 1),
 ('Categoría:Médicos por país y especialidad', 1)]

In [15]:
top_10 = merge.takeOrdered(10,lambda x: -x[1])

## En limpio

In [16]:
logs = logs_ori.map(lambda x: (x[8],(x[2] if x[2] is not None else x[4]))).filter(lambda x: (x[0] is not None) & (x[1] is not None)).distinct()

In [17]:
contents = contents_ori.map(lambda x: (x[0],1)).filter(lambda x: (x[0] is not None))

In [18]:
merge = logs.join(contents)

In [19]:
merge = merge.map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a + b)

In [23]:
top_10 = merge.takeOrdered(10,lambda x: -x[1])
top_10

[('Wikipedia:Tablón de anuncios de los bibliotecarios/Portal/Archivo/Miscelánea/Actual',
  46),
 ('Wikipedia:Vandalismo en curso', 42),
 ('Wikipedia:Tablón de anuncios de los bibliotecarios/Portal/Archivo/Violaciones de etiqueta/Actual',
  34),
 ('Wikipedia:Informes de error', 34),
 ('Bill Kaulitz', 32),
 ('Wikipedia', 31),
 ('Los Simpson', 31),
 ('Nombre de pila chino', 31),
 ('Don Omar', 31),
 ('Agilidad', 30)]